# 基于协同过滤算法的电影推荐系统

In [2]:
#导入需要用到的库
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
user_file=pd.read_csv('./u.data')
movie_file=pd.read_csv('./u.item.txt',sep='|')

user_df = pd.read_csv('./u.data',sep='\t',names=['user_id','item_id','rating','timestamp'])
movies_df = pd.read_csv('./u.item.txt',encoding='utf-8',sep='|',usecols=['movie_id','movie_title' ,
                                                                            'release date'],
                                                                              names=['movie_id','movie_title' ,
                                                                            'release date','video release date',
                                                                            'IMDb URL','unknown','Action','Adventure',
                                                                            'Animation','Children','Comedy','Crime',
                                                                            'Documentary','Drama','Fantasy','Film-Noir',
                                                                            'Horror','Musical','Mystery',' Romance',
                                                                            'Sci-Fi','Thriller','War','Western'])

In [3]:
user_df.tail(10)

,user_id,item_id,rating,timestamp
99990,806,421,4,882388897
99991,676,538,4,892685437
99992,721,262,3,877137285
99993,913,209,2,881367150
99994,378,78,3,880056976
99995,880,476,3,880175444
99996,716,204,5,879795543
99997,276,1090,1,874795795
99998,13,225,2,882399156
99999,12,203,3,879959583


In [4]:
#查看用户点评数据
user_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 4 columns):
user_id      100000 non-null int64
item_id      100000 non-null int64
rating       100000 non-null int64
timestamp    100000 non-null int64
dtypes: int64(4)
memory usage: 3.1 MB


In [5]:
#查看电影数据
movies_df.tail(10)

,movie_id,movie_title,release date
1672,1673,Mirage (1995),01-Jan-1995
1673,1674,Mamma Roma (1962),01-Jan-1962
1674,1675,"Sunchaser, The (1996)",25-Oct-1996
1675,1676,"War at Home, The (1996)",01-Jan-1996
1676,1677,Sweet Nothing (1995),20-Sep-1996
1677,1678,Mat' i syn (1997),06-Feb-1998
1678,1679,B. Monkey (1998),06-Feb-1998
1679,1680,Sliding Doors (1998),01-Jan-1998
1680,1681,You So Crazy (1994),01-Jan-1994
1681,1682,Scream of Stone (Schrei aus Stein) (1991),08-Mar-1996


In [6]:
movies_df[movies_df['movie_title'].str.contains('Mission')]

,movie_id,movie_title,release date
404,405,Mission: Impossible (1996),22-May-1996


In [7]:
movies_df[movies_df['movie_title'].str.contains('Trans')]

,movie_id,movie_title,release date
425,426,"Transformers: The Movie, The (1986)",01-Jan-1986


In [8]:
n_users = user_df.user_id.unique().shape[0]       #用户总数
n_items = user_df.item_id.unique().shape[0]        #电影总数
data_matrix = np.zeros((n_users,n_items))     #矩阵
for line in user_df.itertuples():               
    data_matrix[line[1]-1, line[2]-1] = line[3] 
data_matrix

array([[5., 3., 4., ..., 0., 0., 0.],
       [4., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [5., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 5., 0., ..., 0., 0., 0.]])

In [9]:
#3计算相似度计算
from sklearn.metrics.pairwise import cosine_similarity
 #电影余弦相似度矩阵
item_similarity=cosine_similarity(data_matrix.T, dense_output=True)     
item_similarity

array([[1.        , 0.40238218, 0.33024479, ..., 0.        , 0.04718307,
        0.04718307],
       [0.40238218, 1.        , 0.27306918, ..., 0.        , 0.07829936,
        0.07829936],
       [0.33024479, 0.27306918, 1.        , ..., 0.        , 0.        ,
        0.09687505],
       ...,
       [0.        , 0.        , 0.        , ..., 1.        , 0.        ,
        0.        ],
       [0.04718307, 0.07829936, 0.        , ..., 0.        , 1.        ,
        0.        ],
       [0.04718307, 0.07829936, 0.09687505, ..., 0.        , 0.        ,
        1.        ]])

推荐系统算法

In [10]:
def movie_recsys(keywords,k):         #keywords：电影名关键词，k：为最接近的k部电影
    keywords=keywords.title() 
    movie_list=[]       #存储结果列表
    try:
        #过滤电影数据集，搜索找到对应的电影的id
        movieid=list(movies_df[movies_df['movie_title'].str.contains(keywords)].movie_id)[0]   
        #获取该电影的余弦相似度数组
        movie_similarity=item_similarity[movieid-1] 
        #返回前K+1个最高相似度的索引位置
        movie_similarity_index=np.argsort(-movie_similarity)[:k+1]   
        

        for i in movie_similarity_index:
            rec_movies=list(item_df[item_df.movie_id==(i+1)].movie_title)   #存储电影名
            rec_movies.append(movie_similarity[i])                          #存储相似度
            rec_movies.append(len(df[df.item_id==(i+1)]))                   #用户数
            rec_movies.append(df[df.item_id==(i+1)]['rating'].mean())       #平均评分分
            movie_list.append(rec_movies)                                  #列表中的元素为列表
    except:
        print('提示：找不到该电影！请检查输入！')
    return movie_list

In [ ]:
def main():
    name=input('请输入电影名称或者关键词：')
    num=int(input('您希望找到多少部相关电影（请输入整数）：'))
    result=movie_recsys(name,num)
    movieid=list(movies_df[movies_df['movie_title']==result[0][0]].movie_id)[0]
    choice=input('是否设置相似度阈值（y/n）')
    if choice=='y':
        threshold=float(input('请设置一个阈值（0到1之间，该值越高电影间相似性越高）：'))
        counter=0
        for i in range(1,num+1):
            if result[i][1]>=threshold:
                counter+=1
                
        if counter==0:
            print('抱歉，找不到符合条件的电影，可能阈值设置过高，请重新设置！')
        else:
            print('{}共有{}个用户评分,平均评分为{}(评分上限为5分)'.format(result[0][0],len(user_df[user_df.item_id==movieid]),
                                                          "%.5f" %user_df[user_df.item_id==movieid]['rating'].mean()))
                                                                  
            print('经过阈值处理，与{}最相似的前{}部是（剔除了{}部）：'.format(result[0][0],counter,num-counter))
            print()
            for i in range(1,counter+1):
                print(result[i][0],',相似度为:{}, 共有{}个用户评分，平均评分为{}(评分上限为5分)'.format(
                                                                            "%.6f" %result[i][1],
                                                                               result[i][2],
                                                                          "%.5f" %result[i][3]))
                
    else:
        print('{}共有{}个用户评分，平均评分为{}(评分上限为5分)'.format(result[0][0],len(user_df[user_df.item_id==movieid]),
                                                            "%.5f" %user_df[user_df.item_id==movieid]['rating'].mean()))
        print('与{}最相似的前{}部是：'.format(result[0][0],num))
        print()
        for i in range(1,num+1):
            print(result[i][0],',相似度为:{}, 共有{}个用户评分，平均评分为{}(评分上限为5分)'.format(
                                                                                    "%.6f" % result[i][1],
                                                                                       result[i][2],
                                                                                  "%.5f" % result[i][3]))

    c=input('是否继续使用？（y/n）：')
    if c=='y':
        main() 
    else:
        print('谢谢使用！')

main()